<a href="https://colab.research.google.com/github/1000Roses/NLP_course/blob/main/final_LSTM_bai2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install underthesea

In [ ]:
from underthesea import word_tokenize
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
FRIEND_PRE_DATA = list()
VOCAB_SIZE = 0

In [ ]:
path = '/content/drive/MyDrive/NLP/chatbotdata/bạn bè.txt'
with open(path) as frienddata:
  i = 0
  for data in frienddata:
    sent = data.split("__eou__")[:2]
    if len(sent) == 2:
      anq = list()
      for subsent in sent:
        temp = ["sos"] + word_tokenize(subsent) + ["eos"]
        anq.append(temp)
      FRIEND_PRE_DATA.append(anq)
    i+= 1
    if i == 100:
      break


In [ ]:
answers , questions = list(), list()
for question, answer in FRIEND_PRE_DATA:
  answers.append(answer)
  questions.append(question)

In [ ]:
def lower_list(l):
  return [e.lower() for e in l]

In [ ]:
questions = [lower_list(question) for question in questions]
answers   = [lower_list(answer) for answer in answers]

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=True,split=' ')
tokenizer.fit_on_texts(questions + answers)

questions_index = tokenizer.texts_to_sequences(questions) 
answers_index  =  tokenizer.texts_to_sequences(answers) 

word2index = dict(tokenizer.word_index.items())
word2index = { k: v-1 for k,v in word2index.items() }
VOCAB_SIZE = len(word2index) 

index2word = {index : item for item,index in word2index.items()}

questions_index = pad_sequences(questions_index,VOCAB_SIZE,padding='post', truncating='post')
answers_index = pad_sequences(answers_index,VOCAB_SIZE,padding='post', truncating='post')

In [ ]:
# because start at 1, instead 0, 0 for index2word sync
questions_index = questions_index - 1
answers_index = answers_index - 1

In [ ]:
questions_index[0].shape

(459,)

In [ ]:
answers_index[0]

In [ ]:
questions_index[0]

In [ ]:
index2word[57]

'nhà'

# watch

In [ ]:
# from numpy import array
# from numpy import argmax
# from numpy import array_equal
# from keras.utils import to_categorical
# from keras.models import Model
# from keras.layers import Input
# from keras.layers import LSTM
# from keras.layers import Dense
# from random import randint
 
# # generate a sequence of random integers
# def generate_sequence(length, n_unique):
# 	return [randint(1, n_unique-1) for _ in range(length)]
 
# # prepare data for the LSTM
# def get_dataset(n_in, n_out, cardinality, n_samples):
# 	X1, X2, y = list(), list(), list()
# 	for _ in range(n_samples):
# 		# generate source sequence
# 		source = generate_sequence(n_in, cardinality)
# 		# define padded target sequence
# 		target = source[:n_out]
# 		target.reverse()
# 		# create padded input target sequence
# 		target_in = [0] + target[:-1]
# 		# encode
# 		src_encoded = to_categorical([source], num_classes=cardinality)
# 		tar_encoded = to_categorical([target], num_classes=cardinality)
# 		tar2_encoded = to_categorical([target_in], num_classes=cardinality)
# 		# store
# 		X1.append(src_encoded)
# 		X2.append(tar2_encoded)
# 		y.append(tar_encoded)
# 	return array(X1), array(X2), array(y)
 
# # returns train, inference_encoder and inference_decoder models
# def define_models(n_input, n_output, n_units):
# 	# define training encoder
# 	encoder_inputs = Input(shape=(None, n_input))
# 	encoder = LSTM(n_units, return_state=True)
# 	encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# 	encoder_states = [state_h, state_c]
# 	# define training decoder
# 	decoder_inputs = Input(shape=(None, n_output))
# 	decoder_lstm = LSTM(n_units, return_sequences=True, return_state=True)
# 	decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 	decoder_dense = Dense(n_output, activation='softmax')
# 	decoder_outputs = decoder_dense(decoder_outputs)
# 	model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
# 	# define inference encoder
# 	encoder_model = Model(encoder_inputs, encoder_states)
# 	# define inference decoder
# 	decoder_state_input_h = Input(shape=(n_units,))
# 	decoder_state_input_c = Input(shape=(n_units,))
# 	decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
# 	decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 	decoder_states = [state_h, state_c]
# 	decoder_outputs = decoder_dense(decoder_outputs)
# 	decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)
# 	# return all models
# 	return model, encoder_model, decoder_model
 
# # generate target given source sequence
# def predict_sequence(infenc, infdec, source, n_steps, cardinality):
# 	# encode
# 	state = infenc.predict(source)
# 	# start of sequence input
# 	target_seq = array([0.0 for _ in range(cardinality)]).reshape(1, 1, cardinality)
# 	# collect predictions
# 	output = list()
# 	for t in range(n_steps):
# 		# predict next char
# 		yhat, h, c = infdec.predict([target_seq] + state)
# 		# store prediction
# 		output.append(yhat[0,0,:])
# 		# update state
# 		state = [h, c]
# 		# update target sequence
# 		target_seq = yhat
# 	return array(output)
 
# # decode a one hot encoded string
# def one_hot_decode(encoded_seq):
# 	return [argmax(vector) for vector in encoded_seq]
 
# # configure problem
# n_features = 50 + 1
# n_steps_in = 6
# n_steps_out = 3
# # define model
# train, infenc, infdec = define_models(n_features, n_features, 128)
# train.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
# # generate training dataset
# X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 100000)
# print(X1.shape,X2.shape,y.shape)
# # train model
# train.fit([X1, X2], y, epochs=1)
# # evaluate LSTM
# total, correct = 100, 0
# for _ in range(total):
# 	X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
# 	target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
# 	if array_equal(one_hot_decode(y[0]), one_hot_decode(target)):
# 		correct += 1
# print('Accuracy: %.2f%%' % (float(correct)/float(total)*100.0))
# # spot check some examples
# for _ in range(10):
# 	X1, X2, y = get_dataset(n_steps_in, n_steps_out, n_features, 1)
# 	target = predict_sequence(infenc, infdec, X1, n_steps_out, n_features)
# 	print('X=%s y=%s, yhat=%s' % (one_hot_decode(X1[0]), one_hot_decode(y[0]), one_hot_decode(target)))

# focus

In [ ]:
VOCAB_SIZE

459

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
decoder_target_data =  to_categorical(answers_index)

In [ ]:
decoder_target_data[0]

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [ ]:
encoder_input_data  =  to_categorical(questions_index)

In [ ]:
decoder_input_data = np.array(decoder_target_data, copy= True)

In [ ]:
decoder_input_data[0].shape

(459, 459)

In [ ]:
np.delete(decoder_target_data[0],len(decoder_target_data[0][0]) - 1 , axis=1)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
np.delete(decoder_target_data[0],0 , axis=0)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [ ]:
def process_data(input_data):  
  res = list()
  for i in range(len(input_data)):
    res.append(np.delete(input_data[i], len(input_data[i][0]) - 1, axis = 1))
  res1 = list()
  for i in range(len(input_data)):
    res1.append(np.insert(res[i], -1, 0, axis=1))
  return np.array(res1)

def process_target_data(target_data):
  res = list()
  for i in range(len(target_data)):
    res.append(np.delete(target_data[i], 0, axis = 0))
  res1 = list()
  for i in range(len(target_data)):
    res1.append(np.insert(res[i], len(res[i][0]) - 1 , 0 , axis = 0))
  
  return np.array(res1)
  



In [ ]:
decoder_target_data = process_data(decoder_target_data)
decoder_input_data = process_data(decoder_input_data)
encoder_input_data = process_data(encoder_input_data)

In [ ]:
decoder_target_data = process_target_data(decoder_target_data)

In [ ]:
decoder_input_data[0], decoder_input_data.shape

(array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), (100, 459, 459))

In [ ]:
decoder_target_data[0], decoder_target_data[0].shape

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), (459, 459))

In [ ]:
encoder_input_data[0], encoder_input_data.shape

(array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), (100, 459, 291))

In [ ]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

latent_dim = 300

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, 291))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, 459))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(459, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.compile(loss='categorical_crossentropy',metrics=['acc'],optimizer='adam')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=20,
          epochs=4,
          verbose = 1,
          validation_split=0.2)

Epoch 1/4
4/4 [==============================] - 16s 3s/step - loss: 0.1453 - acc: 0.3787 - val_loss: 0.1715 - val_acc: 8.7146e-04
Epoch 2/4
4/4 [==============================] - 12s 3s/step - loss: 0.1471 - acc: 0.0011 - val_loss: 0.1713 - val_acc: 0.0011
Epoch 3/4
4/4 [==============================] - 12s 3s/step - loss: 0.1414 - acc: 8.5694e-04 - val_loss: 0.1723 - val_acc: 7.6253e-04
Epoch 4/4
4/4 [==============================] - 12s 3s/step - loss: 0.1500 - acc: 7.9884e-04 - val_loss: 0.1763 - val_acc: 8.7146e-04


In [ ]:
# save model
import os.path
if os.path.isfile('models/50epoch.h5') is False:
    model.save('models/50epoch.h5')

In [ ]:
# Load model
from tensorflow.keras.models import load_model
model = load_model('50epoch.h5')

In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=25,
          epochs=10,
          verbose = 1,
          validation_split=0.2)

Epoch 1/10
4/4 [==============================] - 16s 4s/step - loss: 0.1466 - acc: 7.3529e-04 - val_loss: 0.1748 - val_acc: 9.8039e-04
Epoch 2/10
4/4 [==============================] - 13s 3s/step - loss: 0.1440 - acc: 8.4423e-04 - val_loss: 0.1717 - val_acc: 0.0012
Epoch 3/10
4/4 [==============================] - 14s 3s/step - loss: 0.1411 - acc: 8.7146e-04 - val_loss: 0.1693 - val_acc: 0.0012
Epoch 4/10
4/4 [==============================] - 13s 3s/step - loss: 0.1391 - acc: 8.9869e-04 - val_loss: 0.1688 - val_acc: 0.0014
Epoch 5/10
4/4 [==============================] - 13s 3s/step - loss: 0.1388 - acc: 9.8039e-04 - val_loss: 0.1680 - val_acc: 0.0014
Epoch 6/10
4/4 [==============================] - 13s 3s/step - loss: 0.1388 - acc: 0.0010 - val_loss: 0.1683 - val_acc: 0.0014
Epoch 7/10
4/4 [==============================] - 13s 3s/step - loss: 0.1393 - acc: 0.0010 - val_loss: 0.1689 - val_acc: 0.0014
Epoch 8/10
4/4 [==============================] - 13s 3s/step - loss: 0.1392 - a

In [ ]:
decoder_input_data[0].shape

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Test


In [ ]:
states_value = encoder_model.predict(np.array([encoder_input_data[0]]))
num_decoder_tokens = 459

In [ ]:
# Generate empty target sequence of length 1.
target_seq = np.zeros((1, num_decoder_tokens, num_decoder_tokens))
# Populate the first character of target sequence with the start character.
target_seq[0, 0, 0] = 1

In [ ]:
output_tokens, h, c = decoder_model.predict(
        [target_seq] + states_value)

In [ ]:
output_tokens[0].shape

In [ ]:
index2word[4]

In [ ]:
np.argmax(output_tokens[0][9])

In [ ]:
np.argmax(output_tokens[0], axis = 1)

In [ ]:
output_tokens


In [ ]:
encoder_model.predict(np.array([encoder_input_data[1]])) 

In [ ]:
index2word[0]

# Inference

In [ ]:
states_value = encoder_model.predict(np.array([encoder_input_data[6]]))

In [ ]:
num_decoder_tokens = 459

# Generate empty target sequence of length 1.
target_seq = np.zeros((1, num_decoder_tokens, num_decoder_tokens))
# Populate the first character of target sequence with the start character.
target_seq[0, 0, 0] = 1.

# Sampling loop for a batch of sequences
# (to simplify, here we assume a batch of size 1).
stop_condition = False
decoded_sentence = ''
i = 0
while not stop_condition:
    output_tokens, h, c = decoder_model.predict(
        [target_seq] + states_value)

    # Sample a token
    sampled_token_index = np.argmax(output_tokens[0], axis = 0)[i]
    sampled_char = index2word[sampled_token_index]
    decoded_sentence = decoded_sentence + sampled_char + " "

    # Exit condition: either hit max length
    # or find stop character.
    if (len(decoded_sentence.split(" ")) > 100):  # i think set sampled_char = eos instead limited length
        stop_condition = True

    # Update the target sequence (of length 1).
    target_seq = np.zeros((1, num_decoder_tokens, num_decoder_tokens)) # i think remove thís line
    target_seq[0, sampled_token_index, sampled_token_index] = 1.
    i += 1

    # Update states
    states_value = [h, c]



In [ ]:
output_tokens.shape

In [ ]:
encoder_input_data[2]

In [ ]:
decoded_sentence

'sos sos thế được còn còn để để sách bạn bè bạn bè ngày ngày ngày sở thích sở thích sở thích sở thích sos sos sos eos cậu cậu cậu cậu mày nữa nữa mới mới mới nơi nơi này phải phải phải quán ngoài " dạo bữa thấy nữ đầu sos ? ? xem xem ? ? ? bạn mình mình , không thích thích thích . . sos rồi rồi chưa là người cậu vậy vậy vậy vậy vậy vậy vậy thế tỏ tình tỏ tình nơi mỗi mỗi mỗi ngày ngày ngày ngày sở thích sở thích '

In [ ]:
questions[6]

['sos', 'bao lâu', 'rồi', 'chưa', 'về', 'quê', 'eos']